In [ ]:
!pip install openai -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 14.7 MB/s eta 0:00:00


In [ ]:
import openai
openai.api_key = ""

#Set threshold value

In [ ]:
t = 0.1

In [ ]:
# problem = "How do you serve a single slice of pizza to a 100 people"
# problem = "What is the easiest hack to manage your time?"
# problem = "How do you make a spend a billion dollars in a single day?"
problem = "I have $100 in the bank. What is the one thing that I should buy for myself?"
# problem = "I have $100 in the bank. Discuss vaious ways I can use it to help people in need."
# problem = "What's the one thing that I should buy right now, that I will be thankful for in the future?"

In [ ]:
system_content = f"""
You are a logical and problem solver genius. Given a problem you always find the best and most simple and obvious solution.
Always break down the problem, objects, numbers and logic before starting to solve the problem in a step-by-step way.
Find the simplest and most efficient way to solve the problem. 
For any given Problem, please consider three solutions. Make sure that the context of the new solutions is unique with each other. 
Don't repeat the solutions.  For each solution, generate a score to measure the efficiency to solve the problem. Then compare the scores to find out the best solution. Explain the best solution step-by-step. 
Here is the problem: {problem}

Here is an example of how to generate a score to the solution.
Don't give the answer in bullet points, but in paragraphs.

Score: <score>
Answer: <answer>
The score and answer should be inline.

Score should be on a scale of 0 to 10. Higher the value, greater the priority to that answer. 

Example problem: How to solve global warming?
Score 1: 8.5
Answer 1: Implement a combination of policies and initiatives that promote clean energy, reduce greenhouse gas emissions, and encourage sustainable practices. This can include transitioning to renewable energy sources like solar, wind, and hydroelectric power; improving public transportation and promoting electric vehicles; increasing energy efficiency in buildings and appliances; encouraging reforestation and afforestation efforts; and implementing carbon pricing to incentivize businesses and individuals to reduce their emissions.
Score 2: 6.5
Answer 2: Drive innovation and advancements in carbon capture and storage technology. Expand research and development funding to find new ways of capturing and storing excess carbon dioxide from the atmosphere, allowing us to mitigate the effects of global warming without drastic changes to our current way of life. However, this solution alone cannot fully address the problem and should be combined with other efforts to reduce overall emissions.
Score 3: 3.5
Answer 3: Geoengineering or climate engineering to counteract global warming. This could involve intentional large-scale modifications to the Earth's environment, such as solar radiation management or ocean fertilization. Although these ideas could potentially help address global warming, they pose significant risks and uncertainties, including unintended ecosystem disruptions and geopolitical conflicts arising from differing interests in climate modification technologies.
"""

In [ ]:
user_content = """
Score 1:
Answer 1:

Score 2:
Answer 2:

Score 3:
Answer 3:

"""

In [ ]:
winning_scores = []
winning_answers = []

def percentage_error(y1, y2):
    return abs(y1 - y2) / y1 * 100

In [ ]:
import re
for i in range(0, 10):
  try:
    m1 = [{"role": "system", "content": f"{system_content}"},
          {"role": "user", "content": f"{user_content}"}]        
    result = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        max_tokens = 1000,
        temperature =0.8,
        messages=m1)
    response = result["choices"][0]['message']['content']
    break
  except:
    continue


pattern = r'Score 1: (.|\n)*'

match = re.search(pattern, response)

if match:
  trimmed_text = match.group()
  text = trimmed_text
else:
  text = response


loss = 10


score_pattern = r"Score \d+: (.+)"
answer_pattern = r"Answer \d+: (.+)"

scores = re.findall(score_pattern, text)
answers = re.findall(answer_pattern, text)
final_score_index = scores.index(max(scores))
final_score = max(scores)
final_answer = answers[final_score_index]

winning_scores.append(final_score)
winning_answers.append(final_answer)
print(final_answer)

i = 0
while loss>t:
  try:

    system_content2 = f"""
    You are a logical and problem solver genius. Given a problem you always find the best and most simple and obvious solution.
    Always break down the problem, objects, numbers and logic before starting to solve the problem in a step-by-step way.
    Find the simplest and most efficient way to solve the problem. 
    For any given Problem, please generate two more solutions. Make sure that the context of the new solutions is unique with each other.
    Don't repeat any solutions. Don't generate solutions that are alike to each other. The new solutions have to be completely unique.
    For each solution, generate a score to measure the efficiency to solve the problem. Then compare the scores to find out the best solution. Explain the best solution step-by-step. 
    Here is the problem: {problem}
    Here is a possible solution: {winning_answers[i]}

    Here is an example of how to generate a score to the solution.
    Don't give the answer in bullet points, but in paragraphs.

    Score: <score>
    Answer: <answer>
    The score and answer should be inline.

    Score should be on a scale of 0 to 10. Higher the value, greater the priority to that answer. 

    Example problem: How to solve global warming?
    Score 1: 8.5
    Answer 1: Implement a combination of policies and initiatives that promote clean energy, reduce greenhouse gas emissions, and encourage sustainable practices. This can include transitioning to renewable energy sources like solar, wind, and hydroelectric power; improving public transportation and promoting electric vehicles; increasing energy efficiency in buildings and appliances; encouraging reforestation and afforestation efforts; and implementing carbon pricing to incentivize businesses and individuals to reduce their emissions.
    Score 2: 6.5
    Answer 2: Drive innovation and advancements in carbon capture and storage technology. Expand research and development funding to find new ways of capturing and storing excess carbon dioxide from the atmosphere, allowing us to mitigate the effects of global warming without drastic changes to our current way of life. However, this solution alone cannot fully address the problem and should be combined with other efforts to reduce overall emissions.
    Score 3: 3.5
    Answer 3: Geoengineering or climate engineering to counteract global warming. This could involve intentional large-scale modifications to the Earth's environment, such as solar radiation management or ocean fertilization. Although these ideas could potentially help address global warming, they pose significant risks and uncertainties, including unintended ecosystem disruptions and geopolitical conflicts arising from differing interests in climate modification technologies.
    """

    assistant_content = ""
    for answer in winning_answers:
      assistant_content = assistant_content + f"\n{answer}"
        
    m1 = [{"role": "system", "content": f"{system_content2}"},
          {"role": "assistant", "content": f"Previous generated solutions: {assistant_content}"},
        {"role": "user", "content": f"{user_content}"}]        
    result = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        max_tokens = 512,
        temperature =0.3,
        messages=m1)
    response = result["choices"][0]['message']['content']
  except:
    i = i

  pattern = r'Score 1: (.|\n)*'

  match = re.search(pattern, response)

  if match:
    trimmed_text = match.group()
    text = trimmed_text
    
  else:
    text = response
    
  scores = re.findall(score_pattern, text)
  answers = re.findall(answer_pattern, text)
  final_score_index = scores.index(max(scores))
  final_score = max(scores)
  final_answer = answers[final_score_index]
  winning_scores.append(final_score)
  winning_answers.append(final_answer)
  loss = percentage_error(float(winning_scores[i+1]), float(winning_scores[i]))
  print("loss:", loss)
  print("winning answer: ", winning_answers[i+1])
  i = i+1

print("\n\nfinal answer:")
print(winning_answers[-1])
  







The one thing that you should buy for yourself with $100 in the bank depends on your personal needs and priorities. However, one practical and beneficial item to consider is a reusable water bottle. Not only will this help you stay hydrated on-the-go, but it also reduces your environmental impact by minimizing the use of single-use plastic bottles. Additionally, you can find high-quality reusable water bottles for under $100, allowing you to use the remaining money for other important expenses.
loss: 0.0
winning answer:  Invest in a good quality pair of shoes. A comfortable and durable pair of shoes can make a big difference in your daily life, whether you are walking to work, running errands, or exercising. Not only will they provide support and reduce the risk of injury, but they can also boost your confidence and improve your overall well-being. With $100, you can find a high-quality pair of shoes that will last for years to come.


final answer:
Invest in a good quality pair of sho

In [ ]:
#run this cell in case of error to check the ouput to see if it's in the specified format or not. 
text